# Voorbereiding

In [1]:
import os
import json
import asyncio

import traitlets

import numpy as np
import pandas as pd

import ipywidgets as widgets 
from ipywidgets import interact, interact_manual, Layout

import time
from datetime import datetime

from storingsanalyse import StoringsAnalyse

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure

from IPython.display import Markdown, Latex

In [2]:
class LoadedButton(widgets.Button):
    """A button that can holds a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))


# Aparte functie voor het wachten op verandering (aka input van de ME)
def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget.on_click(getvalue, remove=True) 
        # we need to free up the binding to getvalue to avoid an InvalidState error
        # buttons don't support unobserve
        # so use `remove=True` 
    widget.on_click(getvalue)
    return future

In [3]:
""" In the line bellow fill in your API key without < and > """ 
api_key = 'bWF4YWRtaW46R21iQ1dlbkQyMDE5' # provided to you by a Maximo Consultant

""" In the line bellow fill in the desired object structure """
obj_struct = 'MXWO_SND' # name of the Maximo object structure

""" In the line bellow fill in the PO number and the SITEID without the < and > """
query = 'siteid="CT1EN2" and worktype="COR" and reportdate>="2018-01-01T00:00:00-00:00" and reportdate<="2018-03-30T00:00:00-00:00"'

In [4]:
sa = StoringsAnalyse("coentunnel", api_key, obj_struct)

In [5]:
sa.read_staging_file("20210702_12_55_staging_file.xlsx")

In [6]:
sf_data = sa.staging_file_data

### Aanpassen van de staging_file -- DEZE STAP WORDT VERWIJDERD IN DE TOEKOMST
Het opbouwen van deze automatische storingsanalyse vraagt om een ingevulde kolom 'type melding'. Om het process van opbouwen niet te laten stagneren op dit aspect, wordt er hieronder fictive data gegenereerd. Met behulp van de fictieve data wordt het genereren van de verschillende tabellen en grafieken gebouwd.

In [7]:
from random import randrange

lijst_opties = ['Storing', 'Incident', 'Preventief', 'Onterecht']
random_ranges = [lijst_opties[randrange(0, 4, 1)] for _ in range(len(sf_data['type melding (Storing/Incident/Preventief/Onterecht)']))]
sf_data.loc[:, 'type melding (Storing/Incident/Preventief/Onterecht)'] = random_ranges

In [8]:
lijst_opties = ['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', np.nan]
random_ranges = [lijst_opties[randrange(0, len(lijst_opties), 1)] for _ in range(len(sf_data['probleem code']))]
sf_data.loc[:, 'probleem code'] = random_ranges

In [9]:
lijst_opties = ['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', np.nan]
random_ranges = [lijst_opties[randrange(0, len(lijst_opties), 1)] for _ in range(len(sf_data['oorzaak code']))]
sf_data.loc[:, 'oorzaak code'] = random_ranges

In [10]:
lijst_opties = ['S01', 'S02', 'S03', 'S04', 'S05', 'S06', 'S07', 'S08', np.nan]
random_ranges = [lijst_opties[randrange(0, len(lijst_opties), 1)] for _ in range(len(sf_data['oplos code']))]
sf_data.loc[:, 'oplos code'] = random_ranges

In [11]:
# sf_data.iloc[:5, 15:25]

In [12]:
# changing the di_numbers from '45-10' to '45'
new_num = []
for num in sf_data['sbs']:
    if num is np.nan:
        new_num.append(num)
    else:
        new_num.append(sa._isolate_di_number(num))

sf_data.loc[:, 'sbs'] = new_num

In [13]:
# sf_data.iloc[:5, 5:15]

In [14]:
sa.split_staging_file()

'Data available through the use of StoringsAnalyse.meldingen and StoringsAnalyse.storingen'

# Analyse

## Aantallen meldingen

### Aantal meldingen per maand

In [15]:
totaal_aantal_meldingen = len(sf_data.index)

meldingen_per_maand = sf_data['month_number'].value_counts()

gemiddelde_per_maand = sum(meldingen_per_maand) / len(meldingen_per_maand)

num_maand_max_meldingen = [meldingen_per_maand.index[meldingen_per_maand == max(meldingen_per_maand)][0]]
maand_max = sa._month_num_to_name(month_num=num_maand_max_meldingen)

num_maand_min_meldingen = [meldingen_per_maand.index[meldingen_per_maand == min(meldingen_per_maand)][0]]
maand_min = sa._month_num_to_name(month_num=num_maand_min_meldingen)

data_2019 = [key for key in sa.metadata.meldingen().keys() if '2019' in key]
meldingen_2019 = sa.metadata.sum_values(dictionary=sa.meldingen, keys=data_2019)

jaarlijks_gemiddelde = sa.metadata.avg_yearly(dictionary=sa.metadata.meldingen(), exclude_year='2020')

maanden = sa.metadata.get_month_list(exclude_year='2020')
maandelijks_gemiddelde = sa.metadata.avg_monthly(dictionary=sa.metadata.meldingen(), exclude_keys=maanden)

kwartaal_gemiddelde = sa.metadata.avg_quarterly(dictionary=sa.metadata.meldingen())

In [16]:
tekst = f"""
Om te kunnen bepalen of een trend waarneembaar is in het aantal meldingen per 
maand, wordt als onderdeel van deze rapportage een grafiek toegevoegd. Zie 
bijlage 1: “Aantal meldingen per maand”.

Uit de grafiek valt het volgende te constateren:

• Het totaal aantal meldingen in Q1 2021 : **{totaal_aantal_meldingen}** 

• Het gemiddelde aantal meldingen per maand : **{gemiddelde_per_maand}** 

• Hoogste aantal meldingen in de maand **{maand_max}** : **{max(meldingen_per_maand)}**

• Laagste aantal meldingen in de maanden **{maand_min}** : **{min(meldingen_per_maand)}**

• Het gemiddelde aantal meldingen per maand vanaf **{sa.start_date}**: **{maandelijks_gemiddelde}**

• Het gemiddelde aantal meldingen per kwartaal vanaf **{sa.start_date}**: **{kwartaal_gemiddelde}**
"""
display(Markdown(tekst))


Om te kunnen bepalen of een trend waarneembaar is in het aantal meldingen per 
maand, wordt als onderdeel van deze rapportage een grafiek toegevoegd. Zie 
bijlage 1: “Aantal meldingen per maand”.

Uit de grafiek valt het volgende te constateren:

• Het totaal aantal meldingen in Q1 2021 : **99** 

• Het gemiddelde aantal meldingen per maand : **33.0** 

• Hoogste aantal meldingen in de maand **Februari** : **45**

• Laagste aantal meldingen in de maanden **Maart** : **24**

• Het gemiddelde aantal meldingen per maand vanaf **01-2016**: **35.98412698412698**

• Het gemiddelde aantal meldingen per kwartaal vanaf **01-2016**: **107.95238095238095**


In [17]:
huidige_q = 'Q1'
huidig_jaar = '2021'

# Aantal meldingen in voorgaande q
voorgaande_q = sa.quarter_sequence.get_prev_val(huidige_q)
voorgaand_jaar = str(int(huidig_jaar) - 1)
mlist = sa.metadata.get_keys(dictionary=sa.metadata.meldingen(), containing_quarter=[voorgaande_q], containing_year=[voorgaand_jaar])
meldingen_gefilterd = sa.metadata.filter_dictionary_keys(dictionary=sa.metadata.meldingen(), keys=mlist)
totaal_meldingen_voorgaand_kwartaal = sa.metadata.sum_values(meldingen_gefilterd)

# Aantal meldingen in zelfde q voorgaand jaar
mlist = sa.metadata.get_keys(dictionary=sa.metadata.meldingen(),containing_quarter=[huidige_q], containing_year=[voorgaand_jaar])
meldingen_gefilterd = sa.metadata.filter_dictionary_keys(dictionary=sa.metadata.meldingen(), keys=mlist)
totaal_meldingen_zelfde_kwartaal = sa.metadata.sum_values(meldingen_gefilterd)

In [18]:
tekst = f"""
In **{huidige_q}** **{voorgaand_jaar}** waren in totaal **{totaal_meldingen_zelfde_kwartaal}** meldingen gemaakt. In **{huidige_q}** **{huidig_jaar}** zijn er **{totaal_aantal_meldingen-totaal_meldingen_zelfde_kwartaal}** meldingen 
meer t.o.v. **{huidige_q}** **{voorgaand_jaar}**. 
 
In **{voorgaande_q}** **{voorgaand_jaar}** waren in totaal **{totaal_meldingen_voorgaand_kwartaal}** meldingen gemaakt. In **{huidige_q}** **{huidig_jaar}** zijn er **{totaal_aantal_meldingen-totaal_meldingen_voorgaand_kwartaal}** meldingen 
meer t.o.v. **{voorgaande_q}** **{voorgaand_jaar}**. 
"""
display(Markdown(tekst))


In **Q1** **2020** waren in totaal **60** meldingen gemaakt. In **Q1** **2021** zijn er **39** meldingen 
meer t.o.v. **Q1** **2020**. 
 
In **Q4** **2020** waren in totaal **79** meldingen gemaakt. In **Q1** **2021** zijn er **20** meldingen 
meer t.o.v. **Q4** **2020**. 


### Aantal meldingen per subsysteem 

In [19]:
df = sa.meldingen.copy()
# unieke types vastlegen
unique_types = list(df.loc[:, 'sbs'].unique())

sbs_count = df.loc[:, 'sbs'].value_counts()

In [20]:
input_threshhold = widgets.IntSlider(
    value=0,
    min=0,
    max=sbs_count.max(),
    step=1,
    description='Drempelwaarde:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=dict(description_width='initial')
)

display(input_threshhold)

IntSlider(value=0, continuous_update=False, description='Drempelwaarde:', max=10, style=SliderStyle(descriptio…

In [21]:
sbs_to_process = [x for x in sbs_count.index if sbs_count.at[x] >= input_threshhold.value]

r2p = list()
for sbs in sbs_to_process:
    meldingen_per_sbs = sbs_count[sbs]
    percentage_storing = round((meldingen_per_sbs / sum(sbs_count)) * 100, 2)
    line = f"{sa._get_breakdown_description(sbs)[0]}\t- {meldingen_per_sbs} meldingen ({percentage_storing}% van het totale aantal meldingen)"
    r2p.append(line)

In [22]:
# notification type
ntypes = list(df.loc[:, 'type melding (Storing/Incident/Preventief/Onterecht)'].unique())
ntype_count = df.loc[:, 'type melding (Storing/Incident/Preventief/Onterecht)'].value_counts()
n2p = list()
for n in ntypes:
    line = f"{ntype_count[n]} meldingen zijn gecategoriseerd als {n}."
    n2p.append(line)

In [23]:
# '\' is not allowed as character in a f-string like bellow. reason for newline and tab
newline = '\n'
tab = '\t'

tekst=f"""
Er wordt en Pareto analyse gemaakt van het totaal aantal meldingen per subsysteem. Deze is toegevoegd als bijlage 1. 
 
Uit de pareto blijkt dat in **{huidige_q}** **{voorgaand_jaar}** een totaal van **{totaal_aantal_meldingen}** meldingen zijn gemeld, intern 
dan wel extern. Voor het overzicht zijn de meldingen bekeken met **{input_threshhold.value}** of meer 
meldingen. Dit is de top **{len(sbs_to_process)}** en heeft een totaal van {sum(sbs_count[sbs_to_process])}** meldingen van de in totaal 
**{totaal_aantal_meldingen}** (dit is **{round((sum(sbs_count[sbs_to_process])/totaal_aantal_meldingen) * 100, 2)}**% van het totaal). 
Hieronder staan de deelinstallatie**{'s' if len(sbs_to_process) > 1 else ''}**:


{''.join((newline + '**-' + tab + line + '**' + newline for line in r2p))}


De **{totaal_aantal_meldingen}** van **{huidige_q}** **{huidig_jaar}** zijn als volgt onder te verdelen:

{''.join((newline + '**-' + tab + ntype_line + '**' + newline for ntype_line in n2p))}

"""
display(Markdown(tekst))


Er wordt en Pareto analyse gemaakt van het totaal aantal meldingen per subsysteem. Deze is toegevoegd als bijlage 1. 
 
Uit de pareto blijkt dat in **Q1** **2020** een totaal van **99** meldingen zijn gemeld, intern 
dan wel extern. Voor het overzicht zijn de meldingen bekeken met **0** of meer 
meldingen. Dit is de top **17** en heeft een totaal van 43** meldingen van de in totaal 
**99** (dit is **43.43**% van het totaal). 
Hieronder staan de deelinstallatie**s**:



**-	Afsluitbomen (AB)	- 10 meldingen (23.26% van het totale aantal meldingen)**

**-	Verkeerssignaleringssysteem (MTM)	- 7 meldingen (16.28% van het totale aantal meldingen)**

**-	Calamiteiten doorsteek (CaDo)	- 5 meldingen (11.63% van het totale aantal meldingen)**

**-	Brandmeldinstallatie dienstengebouwen	- 4 meldingen (9.3% van het totale aantal meldingen)**

**-	Geluidsbakeninstallatie vluchtdeuren	- 2 meldingen (4.65% van het totale aantal meldingen)**

**-	CCTV-camerasysteem	- 2 meldingen (4.65% van het totale aantal meldingen)**

**-	Verkeersdetectiesysteem (SOS/SDS)	- 2 meldingen (4.65% van het totale aantal meldingen)**

**-	Klimaatinstallatie dienstgebouwen	- 2 meldingen (4.65% van het totale aantal meldingen)**

**-	Brandblusinstallatie tunnel en hulpposten	- 1 meldingen (2.33% van het totale aantal meldingen)**

**-	Tunnelverlichting	- 1 meldingen (2.33% van het totale aantal meldingen)**

**-	Verplaatsbare vangrail (VEVA)	- 1 meldingen (2.33% van het totale aantal meldingen)**

**-	Telefoon en intercominstallatie	- 1 meldingen (2.33% van het totale aantal meldingen)**

**-	Openbare verlichting (OV)	- 1 meldingen (2.33% van het totale aantal meldingen)**

**-	Hoogfrequent installatie (HF-Systeem)	- 1 meldingen (2.33% van het totale aantal meldingen)**

**-	Intern transmissienetwerk	- 1 meldingen (2.33% van het totale aantal meldingen)**

**-	Besturingssysteem	- 1 meldingen (2.33% van het totale aantal meldingen)**

**-	Kantelwalsborden (KW)	- 1 meldingen (2.33% van het totale aantal meldingen)**



De **99** van **Q1** **2021** zijn als volgt onder te verdelen:


**-	33 meldingen zijn gecategoriseerd als Incident.**

**-	22 meldingen zijn gecategoriseerd als Preventief.**

**-	20 meldingen zijn gecategoriseerd als Onterecht.**

**-	24 meldingen zijn gecategoriseerd als Storing.**




## Aantallen storingen

### Aantal storingen per maand

In [24]:
totaal_aantal_storingen = len(sf_data.index)

storingen_per_maand = sf_data['month_number'].value_counts()

gemiddelde_per_maand = sum(storingen_per_maand) / len(storingen_per_maand)

num_maand_max_storingen = [storingen_per_maand.index[storingen_per_maand == max(storingen_per_maand)][0]]
maand_max = sa._month_num_to_name(month_num=num_maand_max_storingen)

num_maand_min_storingen = [storingen_per_maand.index[storingen_per_maand == min(storingen_per_maand)][0]]
maand_min = sa._month_num_to_name(month_num=num_maand_min_storingen)

data_2019 = [key for key in sa.metadata.storingen().keys() if '2019' in key]
storingen_2019 = sa.metadata.sum_values(dictionary=sa.storingen, keys=data_2019)

jaarlijks_gemiddelde = sa.metadata.avg_yearly(dictionary=sa.metadata.storingen(), exclude_year='2020')

maanden = sa.metadata.get_month_list(exclude_year='2020')
maandelijks_gemiddelde = sa.metadata.avg_monthly(dictionary=sa.metadata.storingen(), exclude_keys=maanden)

kwartaal_gemiddelde = sa.metadata.avg_quarterly(dictionary=sa.metadata.storingen())

In [25]:
tekst = f"""
Om te kunnen bepalen of een trend waarneembaar is in het aantal storingen per 
maand, wordt als onderdeel van deze rapportage een grafiek toegevoegd. Zie 
bijlage 1: “Aantal storingen per maand”.

Uit de grafiek valt het volgende te constateren:

• Het totaal aantal storingen in Q1 2021 : **{totaal_aantal_storingen}**

• Het gemiddelde aantal storingen per maand : **{gemiddelde_per_maand}**

• Hoogste aantal storingen in de maand **{maand_max}** : **{max(storingen_per_maand)}**

• Laagste aantal storingen in de maanden **{maand_min}** : **{min(storingen_per_maand)}**

• Het gemiddelde aantal storingen per maand vanaf **{sa.start_date}**: **{maandelijks_gemiddelde}**

• Het gemiddelde aantal storingen per kwartaal vanaf **{sa.start_date}**: **{kwartaal_gemiddelde}**
"""
display(Markdown(tekst))


Om te kunnen bepalen of een trend waarneembaar is in het aantal storingen per 
maand, wordt als onderdeel van deze rapportage een grafiek toegevoegd. Zie 
bijlage 1: “Aantal storingen per maand”.

Uit de grafiek valt het volgende te constateren:

• Het totaal aantal storingen in Q1 2021 : **99**

• Het gemiddelde aantal storingen per maand : **33.0**

• Hoogste aantal storingen in de maand **Februari** : **45**

• Laagste aantal storingen in de maanden **Maart** : **24**

• Het gemiddelde aantal storingen per maand vanaf **01-2016**: **19.19047619047619**

• Het gemiddelde aantal storingen per kwartaal vanaf **01-2016**: **57.57142857142857**


In [26]:
huidige_q = 'Q1'
huidig_jaar = '2021'

# Aantal storingen in voorgaande q
voorgaande_q = sa.quarter_sequence.get_prev_val(huidige_q)
voorgaand_jaar = str(int(huidig_jaar) - 1)
mlist = sa.metadata.get_keys(dictionary=sa.metadata.storingen(), containing_quarter=[voorgaande_q], containing_year=[voorgaand_jaar])
storingen_gefilterd = sa.metadata.filter_dictionary_keys(dictionary=sa.metadata.storingen(), keys=mlist)
totaal_storingen_voorgaand_kwartaal = sa.metadata.sum_values(storingen_gefilterd)

# Aantal storingen in zelfde q voorgaand jaar
mlist = sa.metadata.get_keys(dictionary=sa.metadata.storingen(),containing_quarter=[huidige_q], containing_year=[voorgaand_jaar])
storingen_gefilterd = sa.metadata.filter_dictionary_keys(dictionary=sa.metadata.storingen(), keys=mlist)
totaal_storingen_zelfde_kwartaal = sa.metadata.sum_values(storingen_gefilterd)

In [27]:
tekst = f"""
In **{huidige_q}** **{voorgaand_jaar}** waren in totaal **{totaal_storingen_zelfde_kwartaal}** storingen gemaakt. In **{huidige_q}** **{huidig_jaar}** zijn er **{totaal_aantal_storingen - totaal_storingen_zelfde_kwartaal}** storingen 
meer t.o.v. **{huidige_q}** **{voorgaand_jaar}**. 

In **{voorgaande_q}** **{voorgaand_jaar}** waren in totaal **{totaal_storingen_voorgaand_kwartaal}** storingen gemaakt. In **{huidige_q}** **{huidig_jaar}** zijn er **{totaal_aantal_storingen - totaal_storingen_voorgaand_kwartaal}** storingen 
meer t.o.v. **{voorgaande_q}** **{voorgaand_jaar}**. 
"""
display(Markdown(tekst))


In **Q1** **2020** waren in totaal **31** storingen gemaakt. In **Q1** **2021** zijn er **68** storingen 
meer t.o.v. **Q1** **2020**. 

In **Q4** **2020** waren in totaal **39** storingen gemaakt. In **Q1** **2021** zijn er **60** storingen 
meer t.o.v. **Q4** **2020**. 


### Aantal storingen per subsysteem

In [28]:
df = sa.storingen.copy()
# unieke types vastlegen
unique_types = list(df.loc[:, 'sbs'].unique())

sbs_count = df.loc[:, 'sbs'].value_counts()

sbs_to_process = [x for x in sbs_count.index if sbs_count.at[x] >= input_threshhold.value]

In [29]:
r2p = list()
for sbs in sbs_to_process:
    storingen_per_sbs = sbs_count[sbs]
    percentage_storing = round((storingen_per_sbs / sum(sbs_count)) * 100, 2)
    line = f"{sa._get_breakdown_description(sbs)[0]}\t- {storingen_per_sbs} storingen ({percentage_storing}% van het totale aantal storingen)"
    r2p.append(line)

In [30]:
# '\' is not allowed as character in a f-string like bellow. reason for newline and tab
newline = '\n'
tab = '\t'

tekst = f"""
Er wordt en Pareto analyse gemaakt van het totaal aantal storingen per subsysteem. Deze is toegevoegd als bijlage. 

Uit de pareto blijkt dat in **{huidige_q}** **{huidig_jaar}** een totaal van **{totaal_aantal_storingen}** storingen zijn gemeld, intern 
dan wel extern. Voor het overzicht zijn de storingen bekeken met **{input_threshhold.value}** of meer 
storingen. Dit is de top **{len(sbs_to_process)}** en heeft een totaal van **{sum(sbs_count[sbs_to_process])}** storingen van de in totaal 
**{totaal_aantal_storingen}** (dit is **{round((sum(sbs_count[sbs_to_process]) / totaal_aantal_storingen) * 100, 2)}**% van het totaal). 
Hieronder staan de deelinstallatie**{'s' if len(sbs_to_process) > 1 else ''}**:

{''.join((newline + '**-' + tab + line + '**' + newline for line in r2p))}

In totaal hebben **{len(sbs_count)}** deelsystemen **{min(sbs_count)}** of meerdere storingen gehad in **{huidige_q}** **{huidig_jaar}**.

"""
display(Markdown(tekst))


Er wordt en Pareto analyse gemaakt van het totaal aantal storingen per subsysteem. Deze is toegevoegd als bijlage. 

Uit de pareto blijkt dat in **Q1** **2021** een totaal van **99** storingen zijn gemeld, intern 
dan wel extern. Voor het overzicht zijn de storingen bekeken met **0** of meer 
storingen. Dit is de top **5** en heeft een totaal van **11** storingen van de in totaal 
**99** (dit is **11.11**% van het totaal). 
Hieronder staan de deelinstallatie**s**:


**-	Afsluitbomen (AB)	- 3 storingen (27.27% van het totale aantal storingen)**

**-	Verkeerssignaleringssysteem (MTM)	- 3 storingen (27.27% van het totale aantal storingen)**

**-	Brandmeldinstallatie dienstengebouwen	- 3 storingen (27.27% van het totale aantal storingen)**

**-	Telefoon en intercominstallatie	- 1 storingen (9.09% van het totale aantal storingen)**

**-	Openbare verlichting (OV)	- 1 storingen (9.09% van het totale aantal storingen)**


In totaal hebben **5** deelsystemen **1** of meerdere storingen gehad in **Q1** **2021**.



# Conclusie / aanbeveling

## Algemeen

In [31]:
sbs_count = df.loc[:, 'sbs'].value_counts(dropna=False)

In [32]:
tekst = f"""
Er heeft een analyse van de storingen plaatsgevonden. Uit deze analyse is niet 
naar voren gekomen dat verbeteren aan het onderhoudsplan en/of procedures en/of 
hardware noodzakelijk zijn om het faalgedrag te verbeteren. 
 
Alle meldingen moeten aan een asset / sub niveau van een DI worden gekoppeld. 
Zodat altijd is te herleiden wat precies is gefaald. Aan alle meldingen is DI 
gekoppeld. Aan **{max(sbs_count[sbs_count.index.isnull()].values)}** werkorders zit geen asset gekoppeld. (zie besluit 5). 
 
De meldingen zijn gekoppeld aan een probleem, oorzaak en oplossing. 
 
Vanaf 1 september 2018 heeft een update plaats gevonden van het 
onderhoudsmanagementsysteem. Bij deze update is het invullen van probleem, 
oorzaak en oplossing toegevoegd in het systeem. Vanaf Q4 2018 zal dit ook 
worden meegenomen in de analyse. In de volgende paragrafen staat de uitwerking 
hiervan. Daarbij zie je het aantal van het huidige jaar, het totaal aantal en het 
gemiddelde per Q vanaf Q4 2018.
"""
display(Markdown(tekst))


Er heeft een analyse van de storingen plaatsgevonden. Uit deze analyse is niet 
naar voren gekomen dat verbeteren aan het onderhoudsplan en/of procedures en/of 
hardware noodzakelijk zijn om het faalgedrag te verbeteren. 
 
Alle meldingen moeten aan een asset / sub niveau van een DI worden gekoppeld. 
Zodat altijd is te herleiden wat precies is gefaald. Aan alle meldingen is DI 
gekoppeld. Aan **13** werkorders zit geen asset gekoppeld. (zie besluit 5). 
 
De meldingen zijn gekoppeld aan een probleem, oorzaak en oplossing. 
 
Vanaf 1 september 2018 heeft een update plaats gevonden van het 
onderhoudsmanagementsysteem. Bij deze update is het invullen van probleem, 
oorzaak en oplossing toegevoegd in het systeem. Vanaf Q4 2018 zal dit ook 
worden meegenomen in de analyse. In de volgende paragrafen staat de uitwerking 
hiervan. Daarbij zie je het aantal van het huidige jaar, het totaal aantal en het 
gemiddelde per Q vanaf Q4 2018.


### Probleem

In [33]:
probleem_code_count = sf_data['probleem code'].value_counts(dropna=False).to_dict()
oorzaak_code_count = sf_data['oorzaak code'].value_counts(dropna=False).to_dict()
oplossing_code_count = sf_data['oplos code'].value_counts(dropna=False).to_dict()

meta_probleem = sa.metadata.poo_data()['probleem']
meta_oorzaak = sa.metadata.poo_data()['oorzaak']
meta_oplossing = sa.metadata.poo_data()['oplossing']

poo_beschrijvingen = sa.metadata.contract_info()['POO_codes']


probleem_code_count['Leeg'] = probleem_code_count.pop(np.nan)
oorzaak_code_count['Leeg'] = oorzaak_code_count.pop(np.nan)
oplossing_code_count['Leeg'] = oplossing_code_count.pop(np.nan)

dict_keys(['C11', 'C13', 'C08', 'C04', 'C05', 'C12', 'C16', 'C10', 'C03', 'C09', 'C15', 'C02', 'C06', 'C07', 'C01', 'C14', 'Leeg'])

In [34]:
line_dict = dict()
for code in (k for k in poo_beschrijvingen.keys() if 'P' in k or k == 'Leeg'):
    code_counts = list()
    for q in meta_probleem.keys():
        if code in meta_probleem[q].keys():
            code_counts.append(sa.metadata.sum_values(dictionary=meta_probleem[q], keys=[code]))
        else:
            code_counts.append(0)
    
    totaal = sum(code_counts)
    line = ''.join((newline + '|' + code + '|' + poo_beschrijvingen[code] + '|' + str(probleem_code_count[code]) + '|' + str(totaal) + '|' + 'z' '|'))
    line_dict[code] = line
    
tekst = f"""
|Probleem|Beschrijving|Aantal|Totaal|Gemiddelde|
|--------|------------|------|------|----------|{''.join((line_dict[code] for code in line_dict.keys()))}
"""
display(Markdown(tekst))


|Probleem|Beschrijving|Aantal|Totaal|Gemiddelde|
|--------|------------|------|------|----------|
|P01|Niet beschikbaar|9|107|z|
|P02|Geen / slecht / afwijkend beeld|4|54|z|
|P03|Aangereden|10|44|z|
|P04|In storing|14|118|z|
|P05|Blijft hangen|9|33|z|
|P06|Fatale fout|9|25|z|
|P07|Gewenste stand komt niet overeen (niet mogelijk)|8|9|z|
|P08|Geen communicatie (mogelijk)|4|10|z|
|P09|Oproep actief|7|3|z|
|P10|Niet bedienbaar|11|17|z|
|P11|Overige|6|233|z|
|Leeg|Niet ingevuld|8|162|z|


In [36]:
line_dict = dict()
for code in (k for k in poo_beschrijvingen.keys() if 'C' in k or k == 'Leeg'):
    code_counts = list()
    for q in meta_oorzaak.keys():
        if code in meta_oorzaak[q].keys():
            code_counts.append(sa.metadata.sum_values(dictionary=meta_oorzaak[q], keys=[code]))
        else:
            code_counts.append(0)

    totaal = sum(code_counts)
    line = ''.join((newline + '|' + code + '|' + poo_beschrijvingen[code] + '|' + str(oorzaak_code_count[code]) + '|' + str(totaal) + '|' + 'z' '|'))
    line_dict[code] = line

tekst = f"""
|Oorzaak|Beschrijving|Aantal|Totaal|Gemiddelde|
|-------|------------|------|------|----------|{''.join((line_dict[code] for code in line_dict.keys()))}
"""
display(Markdown(tekst))


|Oorzaak|Beschrijving|Aantal|Totaal|Gemiddelde|
|-------|------------|------|------|----------|
|Leeg|Niet ingevuld|1|180|z|
|C01|Corrosie|4|4|z|
|C02|Mechanische slijtage, - breuk, - vervorming|5|14|z|
|C03|Molest / diefstal|6|3|z|
|C04|Ongedierte|7|0|z|
|C05|Schade|7|12|z|
|C06|Random / software / applicatie fout|5|41|z|
|C07|Thermische vervorming|4|2|z|
|C08|Veroudering|8|99|z|
|C09|Vervuiling|6|7|z|
|C10|Weersomstandigheden|6|38|z|
|C11|Externe oorzaak|10|42|z|
|C12|Aanlegkwaliteit|7|3|z|
|C13|Overige|9|309|z|
|C14|Bedienfout|3|20|z|
|C15|Vocht|5|3|z|
|C16|Incident|6|35|z|


In [37]:
line_dict = dict()
for code in (k for k in poo_beschrijvingen.keys() if 'S' in k or k == 'Leeg'):
    code_counts = list()
    for q in meta_oplossing.keys():
        if code in meta_oplossing[q].keys():
            code_counts.append(sa.metadata.sum_values(dictionary=meta_oplossing[q], keys=[code]))
        else:
            code_counts.append(0)

    totaal = sum(code_counts)
    line = ''.join((newline + '|' + code + '|' + poo_beschrijvingen[code] + '|' + str(oplossing_code_count[code]) + '|' + str(totaal) + '|' + 'z' '|'))
    line_dict[code] = line

tekst = f"""
|Oplossing|Beschrijving|Aantal|Totaal|Gemiddelde|
|---------|------------|------|------|----------|{''.join((line_dict[code] for code in line_dict.keys()))}
"""
display(Markdown(tekst))


|oplossing|Beschrijving|Aantal|Totaal|Gemiddelde|
|-------|------------|------|------|----------|
|Leeg|Niet ingevuld|16|180|z|
|S01|Reinigen|12|10|z|
|S02|Vervangen|7|244|z|
|S03|Resetten|8|173|z|
|S04|Afstellen/vastzetten|9|24|z|
|S05|Bijvullen|11|2|z|
|S06|Repareren|8|11|z|
|S07|Smeren / bijvullen / bijwerken|12|2|z|
|S08|Overige|16|166|z|
